In [1]:
import pandas as pd
import numpy as np
from pymongo import MongoClient
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.metrics import mean_squared_error, r2_score

In [66]:
#Createing the connection to the database part 1
mongo = MongoClient(port=27017)
db = mongo['hockey_project_db']
db.list_collection_names()

['2022-2023', '2023-2024']

In [67]:
#creating connections part 2: putting it into a dataframe
data_2022_2023 = db['2022-2023']
answer = list(data_2022_2023.find({}))
df_2022 = pd.json_normalize(answer)

In [68]:
df_2022.head()

,_id,,playerId,season,name,team,position,situation,games_played,minutes/game,...,I_F_mediumDangerShots,I_F_highDangerShots,I_F_lowDangerxGoals,I_F_mediumDangerxGoals,I_F_highDangerxGoals,I_F_lowDangerGoals,I_F_mediumDangerGoals,I_F_highDangerGoals,I_F_unblockedShotAttempts,Line_or_Pair
0,65a0837f1824a1bb36f2b6dd,0,8471817,2022,Ryan Reaves,MIN,R,all,73,9.32,...,29,7,1.16,3.58,1.96,0,4,1,69,3rd line
1,65a0837f1824a1bb36f2b6de,1,8480950,2022,Ilya Lyubushkin,BUF,D,all,68,15.01,...,2,0,1.09,0.28,0.00,1,1,0,52,3rd pair
2,65a0837f1824a1bb36f2b6df,2,8475625,2022,Matt Irwin,WSH,D,all,61,13.04,...,5,0,1.98,0.46,0.00,2,0,0,102,3rd pair
3,65a0837f1824a1bb36f2b6e0,3,8480860,2022,Kevin Bahl,NJD,D,all,42,14.02,...,1,0,1.13,0.08,0.00,1,1,0,47,3rd pair
4,65a0837f1824a1bb36f2b6e1,4,8477952,2022,Robby Fabbri,DET,C,all,28,16.00,...,14,10,1.07,1.73,4.02,1,2,4,53,2nd line


In [69]:
df_2022.drop(['_id', '', 'playerId','season','name','team','situation'], axis='columns',inplace=True)
df_2022.head()

,position,games_played,minutes/game,I_F_xOnGoal,I_F_xGoals,I_F_xRebounds,I_F_primaryAssists,I_F_secondaryAssists,I_F_shotAttempts,I_F_points,...,I_F_mediumDangerShots,I_F_highDangerShots,I_F_lowDangerxGoals,I_F_mediumDangerxGoals,I_F_highDangerxGoals,I_F_lowDangerGoals,I_F_mediumDangerGoals,I_F_highDangerGoals,I_F_unblockedShotAttempts,Line_or_Pair
0,R,73,9.32,49.47,6.70,3.80,5,5,79,15,...,29,7,1.16,3.58,1.96,0,4,1,69,3rd line
1,D,68,15.01,37.10,1.36,1.83,6,6,78,14,...,2,0,1.09,0.28,0.00,1,1,0,52,3rd pair
2,D,61,13.04,69.79,2.44,3.85,1,2,140,5,...,5,0,1.98,0.46,0.00,2,0,0,102,3rd pair
3,D,42,14.02,32.17,1.21,1.81,2,4,79,8,...,1,0,1.13,0.08,0.00,1,1,0,47,3rd pair
4,C,28,16.00,39.69,6.81,2.85,5,4,65,16,...,14,10,1.07,1.73,4.02,1,2,4,53,2nd line


In [6]:
df_2022.to_csv('model_2022-2023.csv')

# Preparing the Data

In [70]:
df_goals_assists = df_2022_no_dummies[['I_F_primaryAssists', 'I_F_secondaryAssists', 'I_F_goals', 'I_F_points']]
df_goals_assists.head()

,I_F_primaryAssists,I_F_secondaryAssists,I_F_goals,I_F_points
0,5,5,5,15
1,6,6,2,14
2,1,2,2,5
3,2,4,2,8
4,5,4,7,16


In [71]:
#Splitting the data
x = df_goals_assists.copy()
x.drop('I_F_points', axis=1, inplace=True)
X = x.values
y = df_goals_assists['I_F_points'].values

In [72]:
#further splitting into test and train data
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [73]:
#creating the scaler variable
scaler = StandardScaler()

In [74]:
X_scalers = scaler.fit(X_train)

In [75]:
# Scaling data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Linear Regression Model

In [76]:
linear_model = LinearRegression()

In [77]:
# Fit the data into the model
linear_model.fit(X_train_scaled, y_train)

LinearRegression()

In [78]:
# Display the model's best fit line formula
print(f"Model's formula: y = {linear_model.intercept_} + {linear_model.coef_[0]}X")

Model's formula: y = 28.337931034482757 + 9.018444569296017X


In [79]:
#Make predictions with the model using X_test_scaled
predicted_y_values = linear_model.predict(X_test_scaled)


In [80]:
# Compute metrics for the linear regression model: score, r2, mse, rmse, std
score = linear_model.score(X_test, y_test, sample_weight=None)
r2 = r2_score(y_test, predicted_y_values)
mse = mean_squared_error(y_test, predicted_y_values)
rmse = np.sqrt(mse)
std = np.std(y_test)

# Print relevant metrics.
print(f"The score is {score}.")
print(f"The r2 is {r2}.")
print(f"The mean squared error is {mse}.")
print(f"The root mean squared error is {rmse}.")
print(f"The standard deviation is {std}.")

The score is -159.02207576962874.
The r2 is 1.0.
The mean squared error is 3.1229132742769953e-30.
The root mean squared error is 1.7671766392404002e-15.
The standard deviation is 25.81642765917697.


# Random Forest

In [81]:
# Create a random forest classifier
rf_model = RandomForestClassifier(n_estimators=1000, random_state=42)

In [82]:
# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)

In [83]:
# Making predictions using the testing data
predictions = rf_model.predict(X_test_scaled)

In [84]:
# Compute metrics for the linear regression model: score, r2, mse, rmse, std
score = rf_model.score(X_test, y_test, sample_weight=None)
r2 = r2_score(y_test, predictions)
mse = mean_squared_error(y_test, predictions)
rmse = np.sqrt(mse)
std = np.std(y_test)

# Print relevant metrics.
print(f"The score is {score}.")
print(f"The r2 is {r2}.")
print(f"The mean squared error is {mse}.")
print(f"The root mean squared error is {rmse}.")
print(f"The standard deviation is {std}.")

The score is 0.0.
The r2 is 0.962397135378254.
The mean squared error is 25.061855670103093.
The root mean squared error is 5.006181745612428.
The standard deviation is 25.81642765917697.


In [85]:
#saving the model
rf_model.save('final_rf_model.h5')

AttributeError: 'RandomForestClassifier' object has no attribute 'save'